<a href="https://colab.research.google.com/github/etgcrog/glue_aws_docker/blob/main/spark_revisao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
!apt-get update -qq
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

In [23]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [24]:
import findspark
findspark.init()

In [98]:
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType, StringType
from pyspark.sql import functions as f

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Iniciando com Spark") \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

In [26]:
data = [('Zeca','35'), ('Eva', '29')]
colNames = ['Nome', 'Idade']

In [27]:
df = spark.createDataFrame(data, colNames)

In [28]:
df.show()

+----+-----+
|Nome|Idade|
+----+-----+
|Zeca|   35|
| Eva|   29|
+----+-----+



In [29]:
df.filter(df.Idade == "35").show(truncate=False)

+----+-----+
|Nome|Idade|
+----+-----+
|Zeca|35   |
+----+-----+



In [49]:
from zipfile import ZipFile

prefix_zip = "/content/drive/MyDrive/Colab Notebooks/data"
prefix_unzip = "/content/data"

names_archives = ['empresas','estabelecimentos','socios']

for archive in names_archives:
  with ZipFile(f"{prefix_zip}/{archive}.zip", 'r') as zip_ref:
    zip_ref.extractall(f"{prefix_unzip}")


In [139]:
def read_parquet(directory):
  return spark.read.options(delimiter=";", inferSchema='True').csv(directory)

def sub_comman_dot(df, col_name):
  return df.withColumn(col_name, f.regexp_replace(col_name, ",", ".").cast(DoubleType()))

def rename_column(df, list_names):
  for index, colName in enumerate(list_names):
    df = df.withColumnRenamed(f"_c{index}", colName)
  return df

def sub_string_date(df, column_name, pattern):
  return df.withColumn(column_name, f.to_date(df[column_name].cast(StringType()), pattern))

In [143]:
df_empresas = read_parquet(f"{prefix_unzip}/empresas")
df_estabelecimentos = read_parquet(f"{prefix_unzip}/estabelecimentos")
df_socios = read_parquet(f"{prefix_unzip}/socios")

In [146]:
#Lista para renomear
empresasColNames = ['cnpj_basico', 'razao_social_nome_empresarial', 'natureza_juridica', 'qualificacao_do_responsavel', 'capital_social_da_empresa', 'porte_da_empresa', 'ente_federativo_responsavel']
estabsColNames = ['cnpj_basico', 'cnpj_ordem', 'cnpj_dv', 'identificador_matriz_filial', 'nome_fantasia', 'situacao_cadastral', 'data_situacao_cadastral', 'motivo_situacao_cadastral', 'nome_da_cidade_no_exterior', 'pais', 'data_de_inicio_atividade', 'cnae_fiscal_principal', 'cnae_fiscal_secundaria', 'tipo_de_logradouro', 'logradouro', 'numero', 'complemento', 'bairro', 'cep', 'uf', 'municipio', 'ddd_1', 'telefone_1', 'ddd_2', 'telefone_2', 'ddd_do_fax', 'fax', 'correio_eletronico', 'situacao_especial', 'data_da_situacao_especial']
sociosColNames = ['cnpj_basico', 'identificador_de_socio', 'nome_do_socio_ou_razao_social', 'cnpj_ou_cpf_do_socio', 'qualificacao_do_socio', 'data_de_entrada_sociedade', 'pais', 'representante_legal', 'nome_do_representante', 'qualificacao_do_representante_legal', 'faixa_etaria']

df_empresas = rename_column(df_empresas, empresasColNames)
df_estabelecimentos = rename_column(df_estabelecimentos, estabsColNames)
df_socios = rename_column(df_socios, sociosColNames)


In [147]:
df_estabelecimentos.show()

+-----------+----------+-------+---------------------------+--------------------+------------------+-----------------------+-------------------------+--------------------------+----+------------------------+---------------------+----------------------+------------------+--------------------+------+-------------------+------------------+--------+---+---------+-----+----------+-----+----------+----------+--------+--------------------+-----------------+-------------------------+
|cnpj_basico|cnpj_ordem|cnpj_dv|identificador_matriz_filial|       nome_fantasia|situacao_cadastral|data_situacao_cadastral|motivo_situacao_cadastral|nome_da_cidade_no_exterior|pais|data_de_inicio_atividade|cnae_fiscal_principal|cnae_fiscal_secundaria|tipo_de_logradouro|          logradouro|numero|        complemento|            bairro|     cep| uf|municipio|ddd_1|telefone_1|ddd_2|telefone_2|ddd_do_fax|     fax|  correio_eletronico|situacao_especial|data_da_situacao_especial|
+-----------+----------+-------+------

In [122]:
# TRATAMENTO DE DADOS
df_empresas = sub_comman_dot(df_empresas, "capital_social_da_empresa")
df_estabelecimentos = sub_string_date(df_empresas, "data", "yyyyMMdd")
# df_estabelecimentos = rename_column()
# df_socios = rename_column()

AnalysisException: ignored

In [111]:
df_empresas.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- razao_social_nome_empresarial: string (nullable = true)
 |-- natureza_juridica: integer (nullable = true)
 |-- qualificacao_do_responsavel: integer (nullable = true)
 |-- capital_social_da_empresa: double (nullable = true)
 |-- porte_da_empresa: integer (nullable = true)
 |-- ente_federativo_responsavel: string (nullable = true)

